# Idaho -- City Sustainability

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import os

from census import Census
from us import states

import plotly.graph_objects as go

In [25]:
c = Census('fb97753783c42ae57fe1a640e38fe04e921e5d1a')

**Greenhouse Gas Data:**

In [26]:
ghg = pd.DataFrame()

for f in os.listdir('../data/2018_data_summary_spreadsheets'):
    temp = pd.read_excel('../data/2018_data_summary_spreadsheets/'+f, sheet_name=0)
    temp['Year'] = f.split('.')[0].split('_')[2]    
    ghg = pd.concat([temp, ghg], sort=False)
    
fips_map = pd.read_excel('../data/fips-codes.xls', sheet_name=0)

fips_map = fips_map[fips_map['Entity Description'] == 'city']

def str_func(x):
    return str(x).zfill(5)

fips_map['FIPS'] = fips_map['FIPS Entity Code'].apply(str_func)
fips_map['City'] = fips_map['GU Name']
fips_map['State'] = fips_map['State Abbreviation']

ghg_mapped = pd.merge(ghg, fips_map, on=['State', 'City'])
total_emissions = ghg_mapped.groupby(['FIPS','Year'])['Total reported direct emissions'].agg('sum').to_frame()

total_emissions.reset_index(inplace=True)

pivot_em = total_emissions.pivot(index='FIPS', columns='Year', values='Total reported direct emissions')

## Get's the 5 largest cities in Idaho

In [27]:
keys = ['NAME' ,'P002001','P002002','P002005','H001001','P013001','H003001','P027001','H005001','H005002','H005003','H005004','H005005','H005006','H005007']

renames_2000 = {
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'P002001': 'Total_Population_2000',
        'P002002':'Total_Urban_Population_2000',
        'P002005':'Total_Rural_Population_2000',
        'H001001': 'Total_Housing_2000',
        'P013001': 'Median_Age_2000',
        'H003001': 'Occupancy_Status_For_Housing_Units_2000',
        'P027001': 'Presence_of_Non-Relatives_2000',
        'H005001': 'Vacancy_Status_2000',
        'H005002': 'For_Rent_2000',
        'H005003': 'Rented_Not_Occupied_2000',
        'H005004': 'For_Sale_Only_2000',
        'H005005': 'Sold_Not_Occupied_2000',
        'H005006': 'For_Seasonal_Recreational_Or_Occasional_Use_2000',
        'H005007': 'For_Migrant_Workers_2000'
}

renames_2010 = {
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'P002001': 'Total_Population_2010',
        'P002002': 'Total_Urban_Population_2010',
        'P002005': 'Total_Rural_Population_2010',
        'H001001': 'Total_Housing_2010',
        'P013001': 'Median_Age_2010',
        'H003001': 'Occupancy_Status_For_Housing_Units_2010',
        'P027001': 'Presence_of_Non-Relatives_2010',
        'H005001': 'Vacancy_Status_2010',
        'H005002': 'For_Rent_2010',
        'H005003': 'Rented_Not_Occupied_2010',
        'H005004': 'For_Sale_Only_2010',
        'H005005': 'Sold_Not_Occupied_2010',
        'H005006': 'For_Seasonal_Recreational_Or_Occasional_Use_2010',
        'H005007': 'For_Migrant_Workers_2010'}

city_2010 = c.sf1.state_place(keys, states.ID.fips, '*', year=2010)
c_pop_2010 = pd.DataFrame.from_records(city_2010)
c_pop_2010_50000 = c_pop_2010.rename(columns=renames_2010)

In [28]:
c_pop_2010_50000.head()

,Total_Housing_2010,Occupancy_Status_For_Housing_Units_2010,Vacancy_Status_2010,For_Rent_2010,Rented_Not_Occupied_2010,For_Sale_Only_2010,Sold_Not_Occupied_2010,For_Seasonal_Recreational_Or_Occasional_Use_2010,For_Migrant_Workers_2010,City_Name,Total_Population_2010,Total_Urban_Population_2010,Total_Rural_Population_2010,Median_Age_2010,Presence_of_Non-Relatives_2010,FIPS,state
0,451.0,451.0,54.0,7.0,0.0,6.0,1.0,19.0,1.0,"Ashton city, Idaho",1127.0,0.0,1127.0,33.4,397.0,03610,16
1,20.0,20.0,9.0,0.0,0.0,1.0,0.0,6.0,0.0,"Banks CDP, Idaho",17.0,0.0,17.0,47.5,11.0,04600,16
2,142.0,142.0,10.0,2.0,0.0,1.0,4.0,0.0,0.0,"Basalt city, Idaho",394.0,0.0,394.0,33.0,132.0,05230,16
3,926.0,926.0,77.0,17.0,3.0,17.0,3.0,19.0,0.0,"Bellevue city, Idaho",2287.0,2144.0,143.0,35.0,849.0,06220,16
4,120.0,120.0,46.0,1.0,0.0,1.0,1.0,42.0,0.0,"Bloomington city, Idaho",206.0,0.0,206.0,46.5,74.0,08560,16


In [29]:
city_2000 = c.sf1.state_place(keys, states.ID.fips, '*', year=2000)
c_pop_2000 = pd.DataFrame.from_records(city_2000)
c_pop_2000_50000 = c_pop_2000.rename(columns=renames_2000)

In [30]:
c_pop_2000_50000.drop(columns=['City_Name', 'state'], inplace=True)

In [31]:
c_pop_2000_50000.head()

,Total_Housing_2000,Occupancy_Status_For_Housing_Units_2000,Vacancy_Status_2000,For_Rent_2000,Rented_Not_Occupied_2000,For_Sale_Only_2000,Sold_Not_Occupied_2000,For_Seasonal_Recreational_Or_Occasional_Use_2000,For_Migrant_Workers_2000,Total_Population_2000,Total_Urban_Population_2000,Total_Rural_Population_2000,Median_Age_2000,Presence_of_Non-Relatives_2000,FIPS
0,1088,1088,119,22,17,2,9,1,68,3193,0,0,27.9,3193,28360
1,20627,20627,1293,556,298,87,84,1,267,51466,0,0,28.8,51466,64090
2,77850,77850,3412,1463,739,302,353,1,554,185787,0,0,32.8,185787,08830
3,9603,9603,640,268,152,40,23,1,156,25967,0,0,28.8,25967,12250
4,4048,4048,184,28,81,18,15,0,42,11085,0,0,35.2,11085,23410


In [32]:
c_pop_2000_50000.set_index('FIPS', inplace=True)
c_pop_2010_50000.set_index('FIPS', inplace=True)

In [33]:
id_join = c_pop_2000_50000.join(c_pop_2010_50000, on='FIPS')
id_join = id_join.join(pivot_em, on='FIPS')

In [34]:
id_join.head()

,Total_Housing_2000,Occupancy_Status_For_Housing_Units_2000,Vacancy_Status_2000,For_Rent_2000,Rented_Not_Occupied_2000,For_Sale_Only_2000,Sold_Not_Occupied_2000,For_Seasonal_Recreational_Or_Occasional_Use_2000,For_Migrant_Workers_2000,Total_Population_2000,...,state,2010,2011,2012,2013,2014,2015,2016,2017,2018
FIPS,,,,,,,,,,,,,,,,,,,,,
28360,1088,1088,119,22,17,2,9,1,68,3193,...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64090,20627,20627,1293,556,298,87,84,1,267,51466,...,16,52242.420,100342.886,88202.054,80940.540,85232.964638,75898.220214,66906.213486,64418.852668,71841.469426
08830,77850,77850,3412,1463,739,302,353,1,554,185787,...,16,165358.696,332.346,3131.146,185263.848,139447.448000,138760.550000,113222.416000,76916.076000,77435.238000
12250,9603,9603,640,268,152,40,23,1,156,25967,...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23410,4048,4048,184,28,81,18,15,0,42,11085,...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
id_join['Total_Population_2000'] = id_join['Total_Population_2000'].astype('i8')

In [36]:
id_join =  id_join.nlargest(5, 'Total_Population_2000')

In [37]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=id_join['City_Name'], y=id_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=id_join['City_Name'], y=id_join['Total_Population_2010']),
    go.Bar(name='2000_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2010']),
    go.Bar(name='2000_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2010']),
])
fig.update_layout(barmode='group')
fig.show()

In [38]:
fig = go.Figure(data=[
    go.Bar(name='2000_age', x=id_join['City_Name'], y=id_join['Median_Age_2000']),
    go.Bar(name='2010_age', x=id_join['City_Name'], y=id_join['Median_Age_2010']),
])
fig.update_layout(barmode='group')
fig.show()

## American Community Servey

In [39]:
i = 0
acs_years = []
for x in range(2012, 2018):
    acs_test = c.acs5.state_place(('NAME',
                                   'B01003_001E',
                                   'B00002_001E',
                                   'B09018_007E',
                                   'B01002_001E'), states.ID.fips, '*', year=x)
    acs_years.append(pd.DataFrame.from_records(acs_test))
    acs_years[i] = acs_years[i].rename(columns={
        'NAME' : 'City_Name',
        'place': 'FIPS',
        'B01003_001E': 'Total_Population_{}'.format(x),
        'B00002_001E': 'Total_Housing_{}'.format(x),
        'B09018_007E': 'Presence_of_Non-Relatives_{}'.format(x),
        'B01002_001E': 'Median_Age_{}'.format(x),
    })
    acs_years[i].set_index('FIPS', inplace=True)
    acs_years[i].drop(columns=['City_Name', 'state'], inplace=True)
    acs_years[i] = acs_years[i].nlargest(5, 'Total_Population_{}'.format(x))
    i = i + 1

In [40]:
for x in acs_years:
    id_join = id_join.join(x)

In [41]:
id_join.head()

,Total_Housing_2000,Occupancy_Status_For_Housing_Units_2000,Vacancy_Status_2000,For_Rent_2000,Rented_Not_Occupied_2000,For_Sale_Only_2000,Sold_Not_Occupied_2000,For_Seasonal_Recreational_Or_Occasional_Use_2000,For_Migrant_Workers_2000,Total_Population_2000,...,Total_Population_2015,Presence_of_Non-Relatives_2015,Total_Housing_2016,Median_Age_2016,Total_Population_2016,Presence_of_Non-Relatives_2016,Total_Housing_2017,Median_Age_2017,Total_Population_2017,Presence_of_Non-Relatives_2017
FIPS,,,,,,,,,,,,,,,,,,,,,
08830,77850,77850,3412,1463,739,302,353,1,554,185787,...,214196.0,933.0,5859.0,35.8,218677.0,591.0,5814.0,36.0,220859.0,356.0
56260,19379,19379,1289,448,440,90,57,3,251,51867,...,86203.0,401.0,1703.0,31.1,87896.0,389.0,1707.0,31.4,89576.0,320.0
64090,20627,20627,1293,556,298,87,84,1,267,51466,...,54549.0,111.0,1797.0,31.4,54592.0,112.0,1763.0,31.6,54658.0,184.0
39700,19771,19771,978,373,200,82,86,0,237,50730,...,58374.0,130.0,1848.0,33.1,58933.0,161.0,1840.0,33.0,59414.0,297.0
52120,12293,12293,464,56,276,45,22,0,65,34919,...,84018.0,272.0,1176.0,35.4,88247.0,374.0,1213.0,35.7,91917.0,262.0


In [42]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=id_join['City_Name'], y=id_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=id_join['City_Name'], y=id_join['Total_Population_2010']),
    go.Bar(name='2012_pop', x=id_join['City_Name'], y=id_join['Total_Population_2012']),
    go.Bar(name='2013_pop', x=id_join['City_Name'], y=id_join['Total_Population_2013']),
    go.Bar(name='2014_pop', x=id_join['City_Name'], y=id_join['Total_Population_2014']),
    go.Bar(name='2015_pop', x=id_join['City_Name'], y=id_join['Total_Population_2015']),
    go.Bar(name='2016_pop', x=id_join['City_Name'], y=id_join['Total_Population_2016']),
    go.Bar(name='2017_pop', x=id_join['City_Name'], y=id_join['Total_Population_2017']),
    go.Bar(name='2000_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2010']),
    go.Bar(name='2012_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2012']),
    go.Bar(name='2013_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2013']),
    go.Bar(name='2014_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2014']),
    go.Bar(name='2015_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2015']),
    go.Bar(name='2016_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2016']),
    go.Bar(name='2017_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2017']),
    go.Bar(name='2000_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2010']),
    go.Bar(name='2012_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2012']),
    go.Bar(name='2013_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2013']),
    go.Bar(name='2014_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2014']),
    go.Bar(name='2015_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2015']),
    go.Bar(name='2016_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2016']),
    go.Bar(name='2017_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2017']),
])
fig.update_layout(barmode='group')
fig.show()

**Plot with emissions data**

In [43]:
fig = go.Figure(data=[
    go.Bar(name='2000_pop', x=id_join['City_Name'], y=id_join['Total_Population_2000']),
    go.Bar(name='2010_pop', x=id_join['City_Name'], y=id_join['Total_Population_2010']),
    go.Bar(name='2012_pop', x=id_join['City_Name'], y=id_join['Total_Population_2012']),
    go.Bar(name='2013_pop', x=id_join['City_Name'], y=id_join['Total_Population_2013']),
    go.Bar(name='2014_pop', x=id_join['City_Name'], y=id_join['Total_Population_2014']),
    go.Bar(name='2015_pop', x=id_join['City_Name'], y=id_join['Total_Population_2015']),
    go.Bar(name='2016_pop', x=id_join['City_Name'], y=id_join['Total_Population_2016']),
    go.Bar(name='2017_pop', x=id_join['City_Name'], y=id_join['Total_Population_2017']),
    go.Bar(name='2000_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2000']),
    go.Bar(name='2010_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2010']),
    go.Bar(name='2012_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2012']),
    go.Bar(name='2013_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2013']),
    go.Bar(name='2014_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2014']),
    go.Bar(name='2015_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2015']),
    go.Bar(name='2016_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2016']),
    go.Bar(name='2017_housing', x=id_join['City_Name'], y=id_join['Total_Housing_2017']),
    go.Bar(name='2000_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2000']),
    go.Bar(name='2010_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2010']),
    go.Bar(name='2012_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2012']),
    go.Bar(name='2013_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2013']),
    go.Bar(name='2014_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2014']),
    go.Bar(name='2015_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2015']),
    go.Bar(name='2016_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2016']),
    go.Bar(name='2017_non-relatives', x=id_join['City_Name'], y=id_join['Presence_of_Non-Relatives_2017']),
    go.Bar(name='2010_direct_emissions', x=id_join['City_Name'], y=id_join['2010']),
    go.Bar(name='2011_direct_emissions', x=id_join['City_Name'], y=id_join['2011']),
    go.Bar(name='2012_direct_emissions', x=id_join['City_Name'], y=id_join['2012']),
    go.Bar(name='2013_direct_emissions', x=id_join['City_Name'], y=id_join['2013']),
    go.Bar(name='2014_direct_emissions', x=id_join['City_Name'], y=id_join['2014']),
    go.Bar(name='2015_direct_emissions', x=id_join['City_Name'], y=id_join['2015']),
    go.Bar(name='2016_direct_emissions', x=id_join['City_Name'], y=id_join['2016']),
    go.Bar(name='2017_direct_emissions', x=id_join['City_Name'], y=id_join['2017']),
    go.Bar(name='2018_direct_emissions', x=id_join['City_Name'], y=id_join['2018'])
])
fig.update_layout(barmode='group')
fig.show()